In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.preprocessing import LabelBinarizer, StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# train = pd.read_csv('./data/rawdata/application_train.csv.zip',compression='zip')
# test = pd.read_csv('./data/rawdata/application_test.csv.zip',compression='zip')

# pre = pd.read_csv('./data/rawdata/previous_application.csv.zip',compression='zip')
bureau = pd.read_csv('./data/rawdata/bureau.csv.zip',compression='zip')
bureau_balance = pd.read_csv('./data/rawdata/bureau_balance.csv.zip',compression='zip')


bureau.csv

All client's previous credits provided by other financial institutions that were reported to Credit Bureau (for clients who have a loan in our sample).

For every loan in our sample, there are as many rows as number of credits the client had in Credit Bureau before the application date.

bureau_balance.csv

Monthly balances of previous credits in Credit Bureau.

This table has one row for each month of history of every previous credit reported to Credit Bureau – i.e the table has (#loans in sample * # of relative previous credits * # of months where we have some history observable for the previous credits) rows.

bureau_balance.csv,STATUS,"Status of Credit Bureau loan during the month (active, closed, DPD0-30,… [
C means closed, 
X means status unknown, 
0 means no DPD, 
1 means maximal did during month between 1-30, 
2 means DPD 31-60,… 
5 means DPD 120+ or sold or written off ] )",

In [3]:
# train (307511, 122)
# test (48744, 121)
bureau.shape
bureau.sort_values('SK_ID_CURR').head(10)

bureau_balance.shape
bureau_balance.head(50)


(1716428, 17)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
248487,100001,5896633,Closed,currency 1,-1572,0,-1329.0,-1328.0,NaN,0,85500.000,0.0,0.000,0.0,Consumer credit,-155,0.0
248485,100001,5896631,Closed,currency 1,-909,0,-179.0,-877.0,NaN,0,279720.000,0.0,0.000,0.0,Consumer credit,-155,0.0
248490,100001,5896636,Active,currency 1,-320,0,411.0,NaN,NaN,0,168345.000,110281.5,NaN,0.0,Consumer credit,-10,9364.5
248489,100001,5896635,Active,currency 1,-49,0,1778.0,NaN,NaN,0,378000.000,373239.0,0.000,0.0,Consumer credit,-16,10822.5
248486,100001,5896632,Closed,currency 1,-879,0,-514.0,-544.0,NaN,0,91620.000,0.0,0.000,0.0,Consumer credit,-155,0.0
248484,100001,5896630,Closed,currency 1,-857,0,-492.0,-553.0,NaN,0,112500.000,0.0,0.000,0.0,Consumer credit,-155,0.0
248488,100001,5896634,Active,currency 1,-559,0,902.0,NaN,NaN,0,337680.000,113166.0,0.000,0.0,Consumer credit,-6,4630.5
675689,100002,6158909,Active,currency 1,-103,0,NaN,NaN,40.500,0,31988.565,0.0,31988.565,0.0,Credit card,-24,0.0
675688,100002,6158908,Closed,currency 1,-645,0,85.0,-36.0,5043.645,0,120735.000,0.0,0.000,0.0,Consumer credit,-34,0.0
675687,100002,6158907,Closed,currency 1,-1121,0,-911.0,-911.0,3321.000,0,19071.000,NaN,NaN,0.0,Consumer credit,-906,0.0


(27299925, 3)

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C
5,5715448,-5,C
6,5715448,-6,C
7,5715448,-7,C
8,5715448,-8,C
9,5715448,-9,0


In [4]:
bureau_balance_features = pd.concat([
    bureau_balance.MONTHS_BALANCE.groupby(bureau_balance.SK_ID_BUREAU).agg(['max','count']),
    bureau_balance.STATUS.groupby(bureau_balance.SK_ID_BUREAU).nunique(),
    bureau_balance.STATUS.groupby(bureau_balance.SK_ID_BUREAU).agg(lambda x: ','.join(set(x))),
    bureau_balance.STATUS.groupby(bureau_balance.SK_ID_BUREAU).agg(lambda x: list(x)[0]),
],axis = 1)
bureau_balance_features.columns = ['MONTHS_BALANCE_max','MONTHS_BALANCE_cnt','STATUS_cntd','STATUS_set','latest_STATUS']
bureau_balance_features = bureau_balance_features.reset_index()

In [5]:
bureau.shape
bureau.tail(10)

(1716428, 17)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
1716418,433007,5057708,Closed,currency 1,-1389,0,-1299.0,-1299.0,0.0,0,334158.435,0.0,0.0,0.0,Consumer credit,-1299,NaN
1716419,352790,5057718,Closed,currency 1,-1808,0,-1596.0,-1625.0,8100.0,0,28248.840,0.0,0.0,0.0,Consumer credit,-1625,NaN
1716420,352790,5057725,Closed,currency 1,-99,0,-83.0,-98.0,NaN,0,27000.000,0.0,0.0,0.0,Consumer credit,-18,NaN
1716421,375755,5057734,Closed,currency 1,-1335,0,-1152.0,-1152.0,NaN,0,195408.000,0.0,NaN,0.0,Consumer credit,-1139,NaN
1716422,375755,5057742,Closed,currency 1,-2648,0,31129.0,-189.0,NaN,0,202500.000,0.0,NaN,0.0,Credit card,-109,NaN
1716423,259355,5057750,Active,currency 1,-44,0,-30.0,NaN,0.0,0,11250.000,11250.0,0.0,0.0,Microloan,-19,NaN
1716424,100044,5057754,Closed,currency 1,-2648,0,-2433.0,-2493.0,5476.5,0,38130.840,0.0,0.0,0.0,Consumer credit,-2493,NaN
1716425,100044,5057762,Closed,currency 1,-1809,0,-1628.0,-970.0,NaN,0,15570.000,NaN,NaN,0.0,Consumer credit,-967,NaN
1716426,246829,5057770,Closed,currency 1,-1878,0,-1513.0,-1513.0,NaN,0,36000.000,0.0,0.0,0.0,Consumer credit,-1508,NaN
1716427,246829,5057778,Closed,currency 1,-463,0,NaN,-387.0,NaN,0,22500.000,0.0,NaN,0.0,Microloan,-387,NaN


In [6]:
bureau = bureau.merge(bureau_balance_features,how='left',on='SK_ID_BUREAU')

bureau_sum = ['CREDIT_DAY_OVERDUE','DAYS_CREDIT_ENDDATE','DAYS_ENDDATE_FACT','DAYS_CREDIT',
               'DAYS_CREDIT_UPDATE','CNT_CREDIT_PROLONG','AMT_CREDIT_SUM','AMT_ANNUITY',
               'AMT_CREDIT_SUM_DEBT','AMT_CREDIT_SUM_LIMIT','AMT_CREDIT_MAX_OVERDUE','MONTHS_BALANCE_max',
              'MONTHS_BALANCE_cnt','STATUS_cntd']

bureau_max = ['AMT_CREDIT_MAX_OVERDUE','CNT_CREDIT_PROLONG', 'DAYS_CREDIT','MONTHS_BALANCE_max','STATUS_cntd']
bureau_countd = ['CREDIT_CURRENCY','CREDIT_ACTIVE','CREDIT_TYPE','STATUS_set','latest_STATUS']

bureau_features = pd.concat([
    bureau.SK_ID_CURR.value_counts(),
    bureau[bureau_sum].groupby(bureau.SK_ID_CURR).sum(),
    bureau[bureau_max].groupby(bureau.SK_ID_CURR).max(),
    bureau[bureau_countd].groupby(bureau.SK_ID_CURR).nunique(),
    bureau[['CREDIT_CURRENCY','CREDIT_ACTIVE','CREDIT_TYPE','latest_STATUS']].groupby(bureau.SK_ID_CURR).agg(lambda x: set(x)),
    bureau[['STATUS_set']].groupby(bureau.SK_ID_CURR).agg(lambda x: set(','.join(x).split(','))),
], axis = 1)

bureau_features.columns = ['bureau_cnt'] + [f+'_sum' for f in bureau_sum] + [f+'_max' for f in bureau_max] + [f+'_cnt' for f in bureau_countd] + [f+'_set' for f in ['CREDIT_CURRENCY','CREDIT_ACTIVE','CREDIT_TYPE','latest_STATUS']] + ['STATUS_set']
bureau_features = bureau_features.reset_index()


In [7]:
bureau_features.shape
bureau_features.head()

(305811, 31)

,SK_ID_CURR,bureau_cnt,CREDIT_DAY_OVERDUE_sum,DAYS_CREDIT_ENDDATE_sum,DAYS_ENDDATE_FACT_sum,DAYS_CREDIT_sum,DAYS_CREDIT_UPDATE_sum,CNT_CREDIT_PROLONG_sum,AMT_CREDIT_SUM_sum,AMT_ANNUITY_sum,...,CREDIT_CURRENCY_cnt,CREDIT_ACTIVE_cnt,CREDIT_TYPE_cnt,STATUS_set_cnt,latest_STATUS_cnt,CREDIT_CURRENCY_set,CREDIT_ACTIVE_set,CREDIT_TYPE_set,latest_STATUS_set,STATUS_set
0,100001,7,0,577.0,-3302.0,-5145,-652,0,1453365.000,24817.5,...,1,2,1,5,3,{currency 1},"{Active, Closed}",{Consumer credit},"{0, C, 1}",{STATUS_set}
1,100002,8,0,-2094.0,-4185.0,-6992,-3999,0,865055.565,0.0,...,1,2,2,4,2,{currency 1},"{Active, Closed}","{Consumer credit, Credit card}","{0, C}",{STATUS_set}
2,100003,4,0,-2178.0,-3292.0,-5603,-3264,0,1017400.500,NaN,...,1,2,2,0,0,{currency 1},"{Active, Closed}","{Consumer credit, Credit card}",{nan},{STATUS_set}
3,100004,2,0,-977.0,-1065.0,-1734,-1064,0,189037.800,NaN,...,1,1,1,0,0,{currency 1},{Closed},{Consumer credit},{nan},{STATUS_set}
4,100005,3,0,1318.0,-123.0,-572,-163,0,657126.000,4261.5,...,1,2,2,3,3,{currency 1},"{Active, Closed}","{Consumer credit, Credit card}","{0, C, X}",{STATUS_set}


In [8]:
bureau_features.to_csv('./data/rawdata/bureau_feature.csv',index=False)